In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import plotnine as p9
import seaborn as sns

<IPython.core.display.Javascript object>

In [3]:
# Request hesitancy data
hesitancy_endpoint = "https://data.cdc.gov/resource/q9mh-h2tw.json?$limit=4000"
r = requests.get(hesitancy_endpoint)
hesitancy_df = pd.DataFrame(r.json())
hesitancy_df.head()

,fips_code,county_name,state,estimated_hesitant,estimated_hesitant_or_unsure,estimated_strongly_hesitant,social_vulnerability_index,svi_category,ability_to_handle_a_covid,cvac_category,...,percent_non_hispanic_asian,percent_non_hispanic_black,percent_non_hispanic_native,percent_non_hispanic_white,geographical_point,state_code,county_boundary,state_boundary,:@computed_region_hjsp_umg2,:@computed_region_skr5_azej
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.24,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,0.0036,0.2697,0,0.6887,"{'type': 'Point', 'coordinates': [-86.844516, ...",AL,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.235,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,0.0061,0.3237,0.0003,0.6263,"{'type': 'Point', 'coordinates': [-86.844516, ...",AL,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,0.0003,0.6938,0,0.2684,"{'type': 'Point', 'coordinates': [-86.844516, ...",AL,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,...,0.0025,0.2354,0,0.6495,"{'type': 'Point', 'coordinates': [-86.844516, ...",AL,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.7,High Vulnerability,0.8,High Concern,...,0.0016,0.0073,0.0005,0.937,"{'type': 'Point', 'coordinates': [-86.844516, ...",AL,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94


<IPython.core.display.Javascript object>

In [4]:
# Extracting political party data from our public github
url = "https://raw.githubusercontent.com/gspiga/STAT141Bfinal/main/data/raw_data.csv"
poli_df = pd.read_csv(url)
poli_df.head()

,Location,Governor Political Affiliation,State Senate Majority Political Affiliation,State House Majority Political Affiliation,State Attorney General Political Affiliation,State Insurance Commissioner Political Affiliation,Footnotes
0,Alabama,Republican,Republican,Republican,Republican,Appointed,NaN
1,Alaska,Republican,Republican,Republican,Appointed,Appointed,NaN
2,Arizona,Republican,Republican,Republican,Republican,Appointed,NaN
3,Arkansas,Republican,Republican,Republican,Republican,Appointed,NaN
4,California,Democrat,Democrat,Democrat,Democrat,Democrat,NaN


<IPython.core.display.Javascript object>

In [5]:
# Changing one column in poli_df to upper case so that we may join them (pandas is case sensitive).
poli_df["Location"] = poli_df["Location"].apply(
    lambda x: x.upper()  # Uppercase all state names
)
poli_df.head()

,Location,Governor Political Affiliation,State Senate Majority Political Affiliation,State House Majority Political Affiliation,State Attorney General Political Affiliation,State Insurance Commissioner Political Affiliation,Footnotes
0,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN
1,ALASKA,Republican,Republican,Republican,Appointed,Appointed,NaN
2,ARIZONA,Republican,Republican,Republican,Republican,Appointed,NaN
3,ARKANSAS,Republican,Republican,Republican,Republican,Appointed,NaN
4,CALIFORNIA,Democrat,Democrat,Democrat,Democrat,Democrat,NaN


<IPython.core.display.Javascript object>

In [21]:
# We join the dataframe
polihes_df = hesitancy_df.merge(poli_df, left_on="state", right_on="Location")

# Have to change decimal values from string to numeric
cols = [
    "estimated_hesitant",
    "estimated_hesitant_or_unsure",
    "estimated_strongly_hesitant",
    "estimated_strongly_hesitant",
    "social_vulnerability_index",
    "ability_to_handle_a_covid",
    "percent_hispanic",
    "percent_non_hispanic_american",
    "percent_non_hispanic_asian",
    "percent_non_hispanic_black",
    "percent_non_hispanic_native",
    "percent_non_hispanic_white",
]
polihes_df[cols] = polihes_df[cols].apply(pd.to_numeric, errors="coerce", axis=1)

<IPython.core.display.Javascript object>

In [7]:
polihes_df.head()

,fips_code,county_name,state,estimated_hesitant,estimated_hesitant_or_unsure,estimated_strongly_hesitant,social_vulnerability_index,svi_category,ability_to_handle_a_covid,cvac_category,...,state_boundary,:@computed_region_hjsp_umg2,:@computed_region_skr5_azej,Location,Governor Political Affiliation,State Senate Majority Political Affiliation,State House Majority Political Affiliation,State Attorney General Political Affiliation,State Insurance Commissioner Political Affiliation,Footnotes
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,...,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,...,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",29,94,ALABAMA,Republican,Republican,Republican,Republican,Appointed,NaN


<IPython.core.display.Javascript object>

We will now form a linear regression model based on SVI, Death rate, ability to handle a covid outbreak, and governor political affiliation to predict hesistancy. 

In [8]:
import statsmodels.api as sm

<IPython.core.display.Javascript object>

In [27]:
X_eth = polihes_df[
    [
        "percent_non_hispanic_american",
        "percent_non_hispanic_asian",
        "percent_hispanic",
        "percent_non_hispanic_black",
        "percent_non_hispanic_native",
        "percent_non_hispanic_white",
    ]
]
X_eth = sm.add_constant(X_eth)
Y = polihes_df["estimated_hesitant"]

model = sm.OLS(Y, X_eth).fit()
predictions = model.predict(X_eth)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     estimated_hesitant   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     103.5
Date:                Wed, 08 Dec 2021   Prob (F-statistic):          2.86e-119
Time:                        22:55:24   Log-Likelihood:                 5477.9
No. Observations:                3142   AIC:                        -1.094e+04
Df Residuals:                    3135   BIC:                        -1.090e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                             0.3897      0.051      7.702      0.000       0.290       0.489
percent_non_hispanic_american    -0.1629      0.055     -2.962      0.003      -0.271      -0.055
percent_non_hispanic_asian       -0.7552      0.067    -11.246      0.000      -0.887      -0.624
percent_hispanic                 -0.3045      0.051     -5.944      0.000      -0.405      -0.204
percent_non_hispanic_black       -0.2031      0.052     -3.942      0.000      -0.304      -0.102
percent_non_hispanic_native      -0.0342      0.163     -0.210      0.833      -0.353       0.285
percent_non_hispanic_white       -0.2583      0.052     -5.011      0.000      -0.359      -0.157
==============================================================================
Omnibus:                      113.032   Durbin-Watson:                   0.296
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              152.412
Skew:                           0.377   Prob(JB):                     8.02e-34
Kurtosis:                       3.772   Cond. No.                         312.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

In [30]:
# Y_sh = polihes_df["estimated_strongly_hesitant"]

# model2 = sm.OLS(Y_sh, X_eth).fit()
# predictions = model2.predict(X_eth)

# model2.summary()

<IPython.core.display.Javascript object>

From our model summary we can see that we get a model for predicting hesistancy with equation $y = 0.3897 - 0.1629x_{nhamer}- -0.7552x_{asian} - 0.3045x_{hispan} - 0.2031x_{black} - 0.0342x_{natam} - 0.2583x_{white}$ We can see that the ethnicity population with the largest influence on vaccine hesitancy is non-hispanic asians with a coefficient of -0.7552. As with all the coefficients in this model, the negative values represent a relationship corresponding to the increase in population of a certain ethnicity/race lowers the estimated hesitancy of a population. 

Another factor to notice in the model is our adjusted R-squared value, which is only 0.164. 